# Marshalling Tests


## Prepare environment

In [ ]:
import os
os.getpid()

In [ ]:
import inspect
import hybridcuda
cures = hybridcuda.initcuda()
hybridcuda.registerheader("hybpython.cuh", os.getcwd() + os.sep + ".." + os.sep + ".." + os.sep + "hybpython.cuh")
assert cures == 0

In [ ]:
class hybridkernel:
    gridDimX = 1
    blockDimX = 1
    shared = 0
    stream = 0
    def __init__(self, func):
        self.hc = hybridcuda.processfunction(func)
        self.hc = hybridcuda.cudajitcode(self.hc)
        self.hc = hybridcuda.ptxlinkcode(self.hc)
        
    def __call__(self, *args):
        self.hc = hybridcuda.launch(self.hc, self.gridDimX,1,1, self.blockDimX,1,1, self.shared,self.stream, *args)

    def __getitem__(self, args):
        if (type(args) != tuple):
            self.grid = args
            return self
        # args is a tuple...
        if (len(args) > 0):
            self.grid = args[0]
        if (len(args) > 1):
            self.block = args[1]
        if (len(args) > 2):
            self.shared = args[2]
        if (len(args) > 3):
            self.stream = args[3]
        return self

#decorator definition
def hybridfunction(func):
    return hybridkernel(func)

#### Passing numpy arrays

In [ ]:
@hybridfunction
def mykernel_copy(N : int, a,b):
    for i in range(0,N):
        b[i] = a[i]

In [ ]:
## prepare some data
import numpy as np
N = 10
a = np.ones(N)
b = np.zeros(N)

mykernel_copy[1,1](N,a,b)

for i in range(0,N):
    assert b[i] == a[i]
print(mykernel_copy.hc["cuda"])

#### Passing built-in value-type without typing

In [ ]:
@hybridfunction
def mykernel_intparam(N, a,b):
    for i in range(0,N):
        b[i] = a[i]

In [ ]:
## prepare some data
import numpy as np
N = 10
a = np.ones(N)
b = np.zeros(N)

mykernel_intparam[1,1](N,a,b)

for i in range(0,N):
    assert b[i] == a[i]

#### Passing lists

In [ ]:
@hybridfunction
def mykernel_listargs(N, a,b):
    for i in range(0,N):
        b[i] = a[i]

In [ ]:
N = 5
a = [1,2,3,4,5]
b = [0,0,0,0,0]

mykernel_listargs[1,1](N,a,b)

for i in range(0,N):
    assert b[i] == a[i]

In [ ]:
b

#### Passing tuples

In [ ]:
@hybridfunction
def mykernel_tupleargs(N, a,b):
    for i in range(0,N):
        b[i] = a[i]

In [ ]:
N = 5
a = (1,2.0,3,4,5)
b = [0,0,0,0,0]

mykernel_tupleargs[1,1](N,a,b)

for i in range(0,N):
    assert b[i] == a[i]

In [ ]:
b

#### Passing arithmetic types

In [ ]:
@hybridfunction
def mykernel_arithtypes(a,b):
    b[0] = a

In [ ]:
b = [0]
mykernel_arithtypes[1,1](1,b)
assert b[0] == 1

In [ ]:
b = [0]
mykernel_arithtypes[1,1](1.0,b)
assert b[0] == 1.0

In [ ]:
b = [0]
mykernel_arithtypes[1,1](1+1j,b)
assert b[0] == (1.0+1.0j)

In [ ]:
b = [0]
mykernel_arithtypes[1,1](float('NaN'),b)

x = float('NaN')
x != x

#### Passing booleans

In [ ]:
@hybridfunction
def mykernel_booltypes(a,b):
    b[0] = a

In [ ]:
b = [0]
mykernel_booltypes[1,1](True,b)
assert b[0] == True
assert type(b[0]) == type(True)

In [ ]:
b[0]